In [1]:
from pyscal.core import System
from pyscal.crystal_structures import Structure
import json
import yaml
import numpy as np
from json import JSONEncoder

In [2]:
class NumpyArrayEncoder(JSONEncoder):
    """
    Encode numpy to dump in json
    """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NumpyArrayEncoder, self).default(obj)
    
def get_angle(vec1, vec2):
    """
    Get angle between two vectors in degrees
    
    Parameters
    ----------
    vec1: list
        first vector
    
    vec2: list
        second vector
    
    Returns
    -------
    angle: float
        angle in degrees
    
    Notes
    -----
    Angle is rounded to two decimal points
    
    """
    return np.round(np.arccos(np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)))*180/np.pi, decimals=2)

def get_coordination(sys):
    sys.find_neighbors(method="cutoff")
    coordination = [len(x) for x in sys.atoms.neighbors.index]
    return coordination

def write_file(outfile, data):
    """
    Write a given dict as json file
    
    Parameters
    ----------
    outfile: string
        name of output file. `.json` will be added to the given file name
    
    data: dict
        input data dict
    
    Returns
    -------
    None
    """
    with open(".".join([outfile, "json"]), "w") as fout:
        json.dump(convert_to_dict(sys), fout, cls=NumpyArrayEncoder)
    #with open(".".join([outfile, "yaml"]), "w") as fout:
    #    yaml.unsafe_dump(convert_to_dict(sys), fout)
        
def convert_to_dict(sys):
    """
    Convert a pyscal System object to data dictionary
    
    Parameters
    ----------
    sys: pyscal System
        input system
    
    Returns
    -------
    info: dict
        dict with parsed information
    """
    info = {}
    
    #not available
    info["latticeparameter"] = sys.atoms._lattice_constant
    info["Coordination"] = get_coordination(sys)
    info["SpaceGroup"] = None #add this on pyscal side (ID and symbol)
    
    #needs to be defined
    if sys.atoms.species[0] is not None:
        info["Element"] = np.unique(sys.atoms.species)
        info["Occupancy"] = sys.atoms.species
    else:
        info["Element"] = np.unique(sys.atoms.types)
        info["Occupancy"] = sys.atoms.types
    
    info["Lattice"] = sys.atoms._lattice 
    info["CellVolume"] = sys.volume
    info["NumberOfAtoms"] = sys.natoms

    info["X_AxisCoordinate"] = np.array(sys.atoms.positions)[:,0]
    info["Y_AxisCoordinate"] = np.array(sys.atoms.positions)[:,1]
    info["Z_AxisCoordinate"] = np.array(sys.atoms.positions)[:,2]

    #available 
    info["FirstAxisComponent"] = sys.box[0]
    info["SecondAxisComponent"] = sys.box[1]
    info["ThirdAxisComponent"] = sys.box[2]
    info["LatticeParameterLengthA"] = np.linalg.norm(sys.box[0])
    info["LatticeParameterLengthB"] = np.linalg.norm(sys.box[1])
    info["LatticeParameterLengthC"] = np.linalg.norm(sys.box[2])
    
    info["LatticeParameterAngleAlpha"] = get_angle(sys.box[0], sys.box[1])
    info["LatticeParameterAngleBeta"] = get_angle(sys.box[1], sys.box[2])
    info["LatticeParameterAngleGamma"] = get_angle(sys.box[2], sys.box[0])    
    
    return info

## POSCAR

In [3]:
sys = System("al_data/Al.poscar", format="poscar")

In [4]:
convert_to_dict(sys)
write_file("dump", convert_to_dict(sys))

## LAMMPS - dump

In [5]:
sys = System("al_data/Al.dump")

In [6]:
convert_to_dict(sys)
write_file("dump2", convert_to_dict(sys))

## CIF/ ASE

In [14]:
from ase.io import read

In [15]:
aseobj = read("al_data/Al.cif", format="cif")

In [16]:
sys = System(aseobj, format="ase")

In [17]:
convert_to_dict(sys)
write_file("dump3", convert_to_dict(sys))

In [18]:
data = convert_to_dict(sys)

## Creating structure from pyscal

In [12]:
struct = Structure()

In [13]:
sys = struct.lattice.b2(lattice_constant=4, element=["Ni", "Al"])

['Ni', 'Al']


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
convert_to_dict(sys)

In [ ]:
sys = struct.element.Si()

In [ ]:
convert_to_dict(sys)

## TODO: pyscal side

- ~~Take a list of elements (optionally)~~ 
- ~~Lattice constant is not multipled to structure creation~~
- Units should always be Angstroms
- ~~Lattice constant is wrong in the above example ; should be 3 - but 9 is used~~
- ~~Always use Cartesian coordinates~~
- Space group

RDFLIB

In [35]:
from rdflib import FOAF, Graph, Literal, Namespace, XSD, RDF, RDFS, URIRef, BNode, SDO
from rdflib import term

In [36]:
CSO = Namespace("https://purls.helmholtz-metadaten.de/disos/cso/")
MDO = Namespace("https://w3id.org/mdo/structure/1.1/")

In [37]:
g = Graph()
#g.bind("cso", CSO)
#g.bind("mdo", MDO)

In [38]:
struct1 = BNode()
struct2 = BNode()

In [39]:
g.add((struct1, RDF.type, MDO.Structure))
g.add((struct2, RDF.type, MDO.Structure))
g.add((struct1, FOAF.name, Literal("struct1")))
g.add((struct2, FOAF.name, Literal("struct2")))


<Graph identifier=N7d9455e95b59454889f7e722bf4b33c4 (<class 'rdflib.graph.Graph'>)>

In [40]:
print(g.serialize(format="json-ld"))

[
  {
    "@id": "_:N158d2ff330d94e43a085b3e337f1d2ab",
    "@type": [
      "https://w3id.org/mdo/structure/1.1/Structure"
    ],
    "http://xmlns.com/foaf/0.1/name": [
      {
        "@value": "struct1"
      }
    ]
  },
  {
    "@id": "_:N4feb78915261486abac67873d757f88a",
    "@type": [
      "https://w3id.org/mdo/structure/1.1/Structure"
    ],
    "http://xmlns.com/foaf/0.1/name": [
      {
        "@value": "struct2"
      }
    ]
  }
]


In [29]:
MDO.Structure

rdflib.term.URIRef('https://w3id.org/mdo/structure/1.1/Structure')

In [22]:
Coordinate_Vector = URIRef("https://w3id.org/mdo/structure/CoordinateVector")
Lattice = URIRef("https://purls.helmholtz-metadaten.de/disos/cso#Lattice")
Structure

In [23]:

g.add((Coordinate_Vector, MDO.X_axisCoordinate, Literal(data["X_AxisCoordinate"])))
g.add((Coordinate_Vector, MDO.Y_axisCoordinate, Literal(data["Y_AxisCoordinate"])))
g.add((Coordinate_Vector, MDO.Z_axisCoordinate, Literal(data["Z_AxisCoordinate"])))

<Graph identifier=N4a938d188e1144a09a7d15b75c83a184 (<class 'rdflib.graph.Graph'>)>

In [24]:
with open("dump-ld.json", "w") as fout:
    fout.write(g.serialize(format="json-ld"))

In [25]:
print(g.serialize(format="json-ld"))

[
  {
    "@id": "https://w3id.org/mdo/structure/CoordinateVector",
    "https://w3id.org/mdo/structure/1.1/X_axisCoordinate": [
      {
        "@value": "[0.         0.         2.01946484 2.01946484]"
      }
    ],
    "https://w3id.org/mdo/structure/1.1/Y_axisCoordinate": [
      {
        "@value": "[0.         2.01946484 0.         2.01946484]"
      }
    ],
    "https://w3id.org/mdo/structure/1.1/Z_axisCoordinate": [
      {
        "@value": "[0.         2.01946484 2.01946484 0.        ]"
      }
    ]
  }
]


- Map the terms (see above)
- Terms for type annotations
- Use BNode to add multiple structures
- Add visualisation
- Space group calculation
- try saving to database
- Query the graph (range queries possible?)
- Make a package out of it
- Extend....